In [11]:
import os, json, ast
import openai
import pandas as pd
from IPython.display import display, JSON

In [9]:
# some utility methods
def search_in(foo, for_string):
    methods = dir(foo)
    return [item for item in methods if for_string.lower() in item.lower()]

In [12]:
# since we are going to use openai's api, we need the open ai key
file_path = '../OPENAI_API_KEY.txt'
with open(file_path, 'r') as file:
    openai.api_key = file.readline()

# update the prices when you update the model you want to use, https://platform.openai.com/docs/pricing
def get_openai_model_price(model_name):
    if(model_name in 'gpt-4o-mini-2024-07-18'):
        return {'name': model_name, 'input_token_price_per_million_usd' : 0.15, 'cached_input_token_price_per_million_usd' : 0.075, 'output_token_price_per_million_usd' : 0.60}
    return None

# calculate price for a response
def get_chat_completion_stats(response):
    response_json = json.loads(response.model_dump_json())
    input_tokens = response_json['usage']['prompt_tokens']
    cached_input_tokens = response_json['usage']['prompt_tokens_details']['cached_tokens']
    output_tokens = response_json['usage']['completion_tokens']

    prices = get_openai_model_price(response_json['model'])
    input_token_price_per_million_usd = prices['input_token_price_per_million_usd']
    cached_input_token_price_per_million_usd = prices['cached_input_token_price_per_million_usd']
    output_token_price_per_million_usd = prices['output_token_price_per_million_usd']

    million = 1000000
    input_tokens_cost = (input_tokens/million)*input_token_price_per_million_usd
    cached_input_tokens_cost = (cached_input_tokens/million)*cached_input_token_price_per_million_usd
    output_token_cost = (output_tokens/million)*output_token_price_per_million_usd

    total_cost = input_tokens_cost + cached_input_tokens_cost + output_token_cost
    output = {
        'input': {
            'rate': input_token_price_per_million_usd,
            'tokens': input_tokens,
            'cost': f'{input_tokens_cost:.10f}'
        },
        'cached_input': {
            'rate': cached_input_token_price_per_million_usd,
            'tokens': cached_input_tokens,
            'cost': f'{cached_input_tokens_cost:.10f}'
        },
        'output': {
            'rate': output_token_price_per_million_usd,
            'tokens': output_tokens,
            'cost': f'{output_token_cost:.10f}'
        },
        'total': {
            'cost': f'{total_cost:.10f}'
        }
    }
    return output


import ipywidgets as widgets
from IPython.display import display, HTML
def display_stats(stats):
    data = stats
    # Create an HTML formatted string for a compact table layout
    html_content = f"""
    <style>
        .usage-table {{
            font-family: Arial, sans-serif;
            font-size: 12px;
            width: 70%;
            border-collapse: collapse;
            margin: 10px auto;
        }}
        .usage-table th, .usage-table td {{
            padding: 6px 10px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }}
        .usage-table th {{
            background-color: #007bff;
            color: white;
            text-align: center;
        }}
        .input-row {{ background-color: #e3f2fd; }}
        .cached-row {{ background-color: #f1f8e9; }}
        .output-row {{ background-color: #ffebee; }}
        .total-row {{ background-color: #007bff; font-weight: bold; }}
    </style>
    
    <table class="usage-table">
        <tr>
            <th>Type</th>
            <th>Rate ($/1M)</th>
            <th>Tokens</th>
            <th>Cost ($)</th>
        </tr>
        <tr class="input-row">
            <td>Input 🔽</td>
            <td>{data['input']['rate']}</td>
            <td>{data['input']['tokens']}</td>
            <td>{data['input']['cost']}</td>
        </tr>
        <tr class="cached-row">
            <td>Cached Input 🔄</td>
            <td>{data['cached_input']['rate']}</td>
            <td>{data['cached_input']['tokens']}</td>
            <td>{data['cached_input']['cost']}</td>
        </tr>
        <tr class="output-row">
            <td>Output 🔼</td>
            <td>{data['output']['rate']}</td>
            <td>{data['output']['tokens']}</td>
            <td>{data['output']['cost']}</td>
        </tr>
        <tr class="total-row">
            <td>Total 💰</td>
            <td>—</td>
            <td>—</td>
            <td>{data['total']['cost']}</td>
        </tr>
    </table>
    """
    
    # Display in Jupyter Notebook
    display(HTML(html_content))

def display_response(response):
    # display(JSON(json.loads(response.model_dump_json())))
    stats = get_chat_completion_stats(response)
    display_stats(stats)

In [13]:
# let's make a simple openai api call to send in a prompt and get back a response
def get_chat_model_completions(messages):
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        temperature = 0,
        max_tokens = 900
    )
    display_response(response)
    return response.choices[0].message.content

test_messages = [
    {'role': 'system', 'content': 'You are a joke teller, you give responses in less than 40 words'},
    {'role': 'user', 'content': 'tell me a joke'}
]

print(get_chat_model_completions(test_messages))

# if we see a response, it means the api-key is still working

def moderation_check(user_input):
    response = openai.moderations.create(input=user_input)
    flagged = response.results[0].flagged
    return 'Flagged' if flagged else 'Not Flagged'

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,30,0.0000045000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,18,0.0000108000
Total 💰,—,—,0.0000153000


Why did the scarecrow win an award? Because he was outstanding in his field!


In [14]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"
    example_user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'low','Multitasking': 'high','Processing speed': 'high','Budget': '150000'}

    system_message = f"""

    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    The value for 'budget' should be a numerical value extracted from the user's response.
    The values currently in the dictionary are only representative values.

    {delimiter}Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised.
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys. The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    {delimiter} Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    You are trying to fill the values of all the keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    Answer "Yes" or "No" to indicate if you understand the requirements and have updated the values for the relevant keys. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n{delimiter}

    {delimiter}Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys. Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    Answer "Yes" or "No" to indicate if you understood all the values for the keys and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.{delimiter}

    {delimiter}Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions. {delimiter}

    Follow the above chain of thoughts and only output the final updated python dictionary. \n


    {delimiter} Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_req}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

def get_laptop_assistant():
    conversation_log = initialize_conversation()
    def laptop_assistant(prompt):
        conversation_log.append({'role': 'user', 'content': prompt})
        response = get_chat_model_completions(conversation_log)
        conversation_log.append({'role': 'assistant', 'content': response})
        return response

    return laptop_assistant

    
    

In [15]:
hey_laptop_assistant = get_laptop_assistant()

In [16]:
hey_laptop_assistant("Hi, I am Suyash. I am looking to buy a laptop")

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1190,0.0001785000
Cached Input 🔄,0.075,1152,0.0000864000
Output 🔼,0.6,63,0.0000378000
Total 💰,—,—,0.0003027000


"Hello Suyash! I'm here to help you find the perfect laptop. To get started, could you please share what you'll primarily be using the laptop for? Are you planning to use it for tasks like gaming, video editing, programming, or general use? Understanding your needs will help me tailor my recommendations."

In [17]:
hey_laptop_assistant("I need a laptop for programming")

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1266,0.0001899000
Cached Input 🔄,0.075,1152,0.0000864000
Output 🔼,0.6,78,0.0000468000
Total 💰,—,—,0.0003231000


'Great! Programming often requires a good balance of processing speed and multitasking capabilities. Could you let me know what kind of programming you do? For example, are you working on web development, software development, data science, or something else? Additionally, do you often run multiple applications or virtual machines at the same time while coding? This will help me understand your multitasking needs better.'

In [18]:
hey_laptop_assistant("web development, software development, yes i will run multiple applications on the machine")

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1366,0.0002049000
Cached Input 🔄,0.075,1280,0.0000960000
Output 🔼,0.6,105,0.0000630000
Total 💰,—,—,0.0003639000


'Thank you for the information, Suyash! Since you are involved in web and software development and will be running multiple applications, it sounds like you will need a laptop with high multitasking capabilities and good processing speed. \n\nNow, could you please let me know how important display quality is for you? For instance, do you prefer a high-resolution display for better visuals, or is that not a priority? Also, do you need a laptop that is lightweight and portable, or will it mostly stay in one place?'

In [20]:
hey_laptop_assistant("my budget is about 40000 indian rupees")

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1676,0.0002514000
Cached Input 🔄,0.075,1536,0.0001152000
Output 🔼,0.6,200,0.0001200000
Total 💰,—,—,0.0004866000


"Thank you for sharing your budget, Suyash. Since your budget is 40,000 INR, I can proceed to fill in the values for the keys in the dictionary. \n\nHere’s the final profile based on your requirements:\n\n- **GPU intensity**: low\n- **Display quality**: low\n- **Portability**: high\n- **Multitasking**: high\n- **Processing speed**: medium\n- **Budget**: 40000\n\nHere is the final updated python dictionary:\n\n```python\n{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': 40000}\n```\n\nPlease note that laptops in this budget range may have limitations in terms of high-end specifications, but I will do my best to find suitable options for you. If you have any further questions or need assistance, feel free to ask!"

In [21]:
response = hey_laptop_assistant("my budget is 40000")
print(response)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,1889,0.0002833500
Cached Input 🔄,0.075,1792,0.0001344000
Output 🔼,0.6,89,0.0000534000
Total 💰,—,—,0.0004711500


Thank you for confirming your budget, Suyash. However, I must inform you that there are no laptops available in that range that meet the requirements for high multitasking and processing speed, especially for programming tasks. 

If you can consider increasing your budget, I would be able to help you find a suitable laptop that meets your needs. Please let me know if you would like to adjust your budget or if you have any other questions!


In [22]:
response = hey_laptop_assistant("i am changing my thought, i need the display quality to be really good")
print(response)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,2000,0.0003000000
Cached Input 🔄,0.075,1920,0.0001440000
Output 🔼,0.6,182,0.0001092000
Total 💰,—,—,0.0005532000


Thank you for the update, Suyash! Since you now prioritize display quality, I will adjust the values accordingly. 

Here’s the revised understanding based on your latest input:

- **GPU intensity**: low (still applicable as programming generally doesn't require high GPU)
- **Display quality**: high (as you want really good display quality)
- **Portability**: high (you need a portable laptop)
- **Multitasking**: high (you will run multiple applications)
- **Processing speed**: medium (still important for programming)
- **Budget**: 40000 (as previously mentioned)

However, I must reiterate that with a budget of 40,000 INR, it may be challenging to find laptops that meet the high display quality requirement along with the other specifications. 

Would you like to consider increasing your budget to find better options? Please let me know!


In [23]:
response = hey_laptop_assistant("ok, increase my budget to 90000")
print(response)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,2198,0.0003297000
Cached Input 🔄,0.075,2176,0.0001632000
Output 🔼,0.6,194,0.0001164000
Total 💰,—,—,0.0006093000


Thank you for increasing your budget to 90,000 INR, Suyash! This will definitely open up more options for laptops that meet your requirements.

Here’s the final profile based on your updated requirements:

- **GPU intensity**: low
- **Display quality**: high
- **Portability**: high
- **Multitasking**: high
- **Processing speed**: medium
- **Budget**: 90000

Here is the final updated python dictionary:

```python
{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': 90000}
```

With this budget, you should be able to find a laptop that offers a great display quality along with good performance for programming tasks. If you have any further questions or need assistance in finding specific models, feel free to ask!


In [24]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt = f"""
    You are a senior evaluator who has an eye for detail.
    You are provided an input. You need to evaluate if the input has the following keys: 'GPU intensity','Display quality','Portability','Multitasking',' Processing speed','Budget'
    Next you need to evaluate if the keys have the the values filled correctly.
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance as stated by user. The value for the key 'budget' needs to contain a number.
    Output a string 'Yes' if the input contains the dictionary with the values correctly filled for all keys.
    Otherwise out the string 'No'.

    Here is the input: {response_assistant}
    Only output a one-word string - Yes/No.
    """

    response = get_chat_model_completions([{ 'role': 'system', 'content': prompt}])
    return response

hey_evaluator = intent_confirmation_layer

In [25]:
evaluator_response = hey_evaluator(response)
print(response)
print(evaluator_response)

Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,363,0.0000544500
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,2,0.0000012000
Total 💰,—,—,0.0000556500


Thank you for increasing your budget to 90,000 INR, Suyash! This will definitely open up more options for laptops that meet your requirements.

Here’s the final profile based on your updated requirements:

- **GPU intensity**: low
- **Display quality**: high
- **Portability**: high
- **Multitasking**: high
- **Processing speed**: medium
- **Budget**: 90000

Here is the final updated python dictionary:

```python
{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': 90000}
```

With this budget, you should be able to find a laptop that offers a great display quality along with good performance for programming tasks. If you have any further questions or need assistance in finding specific models, feel free to ask!
Yes


In [32]:
def dictionary_present(response):
    delimiter = "####"
    user_req = {'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'}
    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.
            Provide the response without any formatting or code block markers.

            Here are some sample input output pairs for better understanding:
            {delimiter}
            input: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input: {{'GPU intensity':     'low', 'Display quality':     'high', 'Portability':    'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90,000'}}
            output: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90000'}}

            input: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'low','Processing speed': 'high','Budget': '200000 INR'
            output: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'low','Budget': '200000'}}
            {delimiter}

            Here is the input {response}

            """

    conversation = [{"role": "system", "content": prompt}]
    # print(conversation)
    response = get_chat_model_completions(conversation)
    return response

hey_python_dictionary_extractor = dictionary_present

In [33]:
specs_dict = hey_python_dictionary_extractor(response)
print(specs_dict)


Type,Rate ($/1M),Tokens,Cost ($)
Input 🔽,0.15,670,0.0001005000
Cached Input 🔄,0.075,0,0.0000000000
Output 🔼,0.6,44,0.0000264000
Total 💰,—,—,0.0001269000


{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '90000'}
